Tools and Agent

# Build Tools and Agent

* LangChain: https://python.langchain.com/docs/modules/agents/


## Configure OpenAI Settings

In [6]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT", "").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Azure AI Search (Cognitive vector store)
vector_store_address: str = os.getenv("AAG_AZURE_SEARCH_SERVICE_ENDPOINT")
vector_store_password: str = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")
# index_name: str = "langchain-vector-arxiv-physics"

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

## Langchain Set up

In [7]:
from langchain.chat_models import AzureChatOpenAI
# Azure OpenAI model
llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo-16k",
    temperature=0,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_type="azure",
    api_key=AZURE_OPENAI_API_KEY,
)

## Prepare Tools for Agent

### Tool: Faiss db retriever

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Get Azure OpenAI embedding
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    deployment=model,
    model=model,
    chunk_size=1,
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_type="azure",
    api_key=AZURE_OPENAI_API_KEY,
)

In [9]:
# Merge all FAISS db
faiss_db = FAISS.load_local("faiss_index_travel", embeddings)

# Set up the retriever we want to use, and then turn it into a retriever tool
faiss_travel_retriever = faiss_db.as_retriever()

In [10]:
# List FAISS index
faiss_db.docstore._dict

{'1ef2b53d-f0a5-41dc-b42c-fdd23e566c54': Document(page_content='12/1/23, 2:22 PM Car rental in Salt Lake City | Expedia\nhttps://www.expedia.com/trips/72674734923624/details/Nzg1NzU5NjUtYzIyZi01NDU1LWJmNGYtYzA5ZWMxMjc3Mjc5O2M0NDMwODUyLWVmZjkt… 1/2Car rental in Salt Lake City\nDec 6, 2023 - Dec 12, 2023\nSign in to see all booking details and manage your trip.\nSign in\nFox\nConfirmation: #WF********\nExpedia itinerary: 7267**********\nReservation details\nPick-up\nWed, Dec 6\n2:30pmDrop-off\nTue, Dec 12\n2:15pm\nPick-up and drop-off instructions\nLocation\nManage booking\nChange, cancel, and support\nPricing and rewards\nMenu\nHelpGet the app EnglishList your propertySupportTrips More travel', metadata={'source': 'data_source\\Car rental in Salt Lake City _ Expedia.pdf', 'page': 0}),
 'cefa2327-9df4-40b8-bbda-b551d93e7ce8': Document(page_content="12/1/23, 2:07 PM JetBlue - Trip review\nhttps://managetrips.jetblue.com/dx/B6DX/#/myb?carReservation=false&hotelReservation=false&locale=en-U

In [11]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents import initialize_agent

# Tool define
faiss_retriever_travel = create_retriever_tool(
    faiss_travel_retriever,
    "Travel information retrieval",
    "This tool is help searching for information in personal travel itineraries, like flight, hotel, Inn, car rental, airport parking",
)

In [12]:
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
faiss_retriever_tools = [
    faiss_retriever_travel,
    ]

# tools = load_tools(["bing-search"], llm=llm)
travel_agent = initialize_agent(
    faiss_retriever_tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8,
)

In [13]:

# Now let's test it out!
airport_parking = travel_agent.run("Could you find out my parking information include address, finally make them in JSON format")



> Entering new AgentExecutor chain...


I need to retrieve parking information and convert it to JSON format.
Action: Travel information retrieval
Action Input: Parking information
Observation: [Document(page_content='12/1/23, 2:03 PM Queens Discount Parking JFK Airport Parking | Way\nhttps://www.way.com/parkingdetails/5069801/Queens-Discount-Parking-JFK-Airport-Parking?cid=5069802 1/5\nQueens Discount Parking JFK Airport Parking\n14707 Guy R Brewer Boulevard, New York, United States ★ ★ ★ ★ ★4 (539 Ratings)\nFavorites\n Been Here 658\n Share\nJFK > JFK Parking Lots > Queens Discount Parking JFK Airport ParkingBooking Overview Reviews Nearby\nBOOKING\nSelect a parking type\nWhy Book With Us?\nBesides the lowest price guarantee, we offer a unique set of benefits that only Way can!\nAccess To Roadside Assistance.\nGet Discounts On Gas, Car Wash, Parking And More When You Check Out.\xa0\nView more benefits\xa0 Outdoor Valet (Daily) From  $7.50\nCheck-In\nWed, Dec 06, 09:00 AMCheck-Out\nTue, Dec 12, 11:30 PM\nAdd to Bag\nOVERVIE

In [15]:
airport_parking

'The parking information is as follows:\n{\n  "address": "14707 Guy R Brewer Boulevard, New York, United States",\n  "rating": "4",\n  "reviews": "539",\n  "type": "Commercial Parking Lot or Garage",\n  "operating_hours": {\n    "Sunday": "24 hours",\n    "Monday": "24 hours",\n    "Tuesday": "24 hours",\n    "Wednesday": "24 hours",\n    "Thursday": "24 hours",\n    "Friday": "24 hours",\n    "Saturday": "24 hours"\n  }\n}'

In [18]:
# Now let's test it out!
flight_info = travel_agent.run("Could you find out my flight information, finally make them in JSON format")



> Entering new AgentExecutor chain...


I need to retrieve the flight information and convert it to JSON format.
Action: Travel information retrieval
Action Input: Flight information
Observation: [Document(page_content="12/1/23, 2:07 PM JetBlue - Trip review\nhttps://managetrips.jetblue.com/dx/B6DX/#/myb?carReservation=false&hotelReservation=false&locale=en-US 1/4Manage your trip\nThere has been a change to your flight. If this new flight does not work for you then use the\n'Change' link below.\nConﬁrmation code\nIHHZZC Ticketed\n\uf152 \uf12c \uf11e\nNew York-JFK\uf10d Salt Lake City CONFIRMED\nDec 6, 2023\n11:00 am\nNew York-JFK\uf10dDec 6, 2023\n02:15 pm\nSalt Lake City5 hrs 15 mins\nNonstop\nBlue Basic\nB6871 A320Operated by JetBlue\nDetails \uf116\nSalt Lake City\uf10d New York-JFK CONFIRMED\nDec 12, 2023\n03:11 pm\nSalt Lake City\uf10dDec 12, 2023\n09:38 pm\nNew York-JFK4 hrs 27 mins\nNonstop\nBlue Basic\nB6872 A320Operated by JetBlueYour flights\uf10f", metadata={'source': 'data_source\\JetBlue - Trip review.pdf', 'pa

In [19]:
flight_info

'The flight information is as follows:\n{\n  "flight1": {\n    "departure": "New York-JFK",\n    "arrival": "Salt Lake City",\n    "date": "Dec 6, 2023",\n    "departureTime": "11:00 am",\n    "arrivalTime": "02:15 pm",\n    "duration": "5 hrs 15 mins",\n    "stops": "Nonstop",\n    "class": "Blue Basic",\n    "flightNumber": "B6871",\n    "operatedBy": "JetBlue"\n  },\n  "flight2": {\n    "departure": "Salt Lake City",\n    "arrival": "New York-JFK",\n    "date": "Dec 12, 2023",\n    "departureTime": "03:11 pm",\n    "arrivalTime": "09:38 pm",\n    "duration": "4 hrs 27 mins",\n    "stops": "Nonstop",\n    "class": "Blue Basic",\n    "flightNumber": "B6872",\n    "operatedBy": "JetBlue"\n  }\n}'

In [20]:
# Now let's test it out!
car_rental = travel_agent.run("Could you find out my car rental information, finally make them in JSON format")



> Entering new AgentExecutor chain...


I need to retrieve the car rental information and convert it to JSON format.
Action: Travel information retrieval
Action Input: Car rental information
Observation: [Document(page_content='12/1/23, 2:22 PM Car rental in Salt Lake City | Expedia\nhttps://www.expedia.com/trips/72674734923624/details/Nzg1NzU5NjUtYzIyZi01NDU1LWJmNGYtYzA5ZWMxMjc3Mjc5O2M0NDMwODUyLWVmZjkt… 1/2Car rental in Salt Lake City\nDec 6, 2023 - Dec 12, 2023\nSign in to see all booking details and manage your trip.\nSign in\nFox\nConfirmation: #WF********\nExpedia itinerary: 7267**********\nReservation details\nPick-up\nWed, Dec 6\n2:30pmDrop-off\nTue, Dec 12\n2:15pm\nPick-up and drop-off instructions\nLocation\nManage booking\nChange, cancel, and support\nPricing and rewards\nMenu\nHelpGet the app EnglishList your propertySupportTrips More travel', metadata={'source': 'data_source\\Car rental in Salt Lake City _ Expedia.pdf', 'page': 0}), Document(page_content='12/1/23, 2:03 PM Queens Discount Parking JFK Airport Parki

In [21]:
car_rental

'The car rental information is as follows:\n\n{\n  "Pick-up": "Wed, Dec 6",\n  "Drop-off": "Tue, Dec 12",\n  "Pick-up and drop-off instructions": "Location",\n  "Confirmation": "#WF********",\n  "Expedia itinerary": "7267**********"\n}'

In [23]:
# Now let's test it out!
hotel = travel_agent.run("Please find out my hotel information")



> Entering new AgentExecutor chain...
I need to retrieve the hotel information from the travel itinerary
Action: Travel information retrieval
Action Input: Hotel
Observation: [Document(page_content='12/1/23, 2:10 PM Reservation Confirmation #84544955 for Fairfield Inn & Suites Salt Lake City South\nhttps://res-marriott.com/H/2/v710000018b156c7fb390364a434b5c38d0/1d17a3a7-0c2e-447c-880c-9f47008d290d/HTML 1/9\nENHANCE YOUR ST AY|SUMMAR Y OF CHARGES |CONT ACT US\nFairfield Inn & Suites Salt Lake City South\n594 W est 4500 South Salt Lake City Utah\n84123 USA\n+1-801-265-9600\nThank you for booking with us, Mr . Jian Gao.\nYou’re ready to go.\nWed, Dec 06, 2023 – Tue, Dec 12, 2023\nConfirmation Number: 84544955\nCheck-In: Wednesday , December 6, 2023 03:00 PM\nCheck-Out: Tuesday , December 12, 2023 11:00 AM\nNumber of rooms 1 Room\nGuests per room 2 Adults\nGuarantee Method Credit Card Guarantee, V isa\nTotal for Stay (all rooms) 811.53 USD\nRoom 1', metadata={'source': 'data_source\\Res

I have retrieved the hotel information from the travel itinerary.
Final Answer: The hotel information is as follows:
Hotel Name: Fairfield Inn & Suites Salt Lake City South
Address: 594 West 4500 South Salt Lake City Utah 84123 USA
Phone Number: +1-801-265-9600
Check-In: Wednesday, December 6, 2023 03:00 PM
Check-Out: Tuesday, December 12, 2023 11:00 AM
Number of rooms: 1 Room
Guests per room: 2 Adults

> Finished chain.


In [24]:
hotel

'The hotel information is as follows:\nHotel Name: Fairfield Inn & Suites Salt Lake City South\nAddress: 594 West 4500 South Salt Lake City Utah 84123 USA\nPhone Number: +1-801-265-9600\nCheck-In: Wednesday, December 6, 2023 03:00 PM\nCheck-Out: Tuesday, December 12, 2023 11:00 AM\nNumber of rooms: 1 Room\nGuests per room: 2 Adults'

In [25]:
trip_name = 'Ski SLC'

# prompt_text = f"""
# Please generate an assessment report on {company_name} based on the following questions and answers in JSON:
# {data}
# and the assessment report should contain the following sections:
# 1. Executive Summary
# 2. Overview of Key Findings
# 3. Summary of Risks Identified
# 4. Overall ratings on the risk level you would give, between 0 and 9 (0 - no risk, 9 - highest risk)
# """


prompt_text = f"""
I am going to plan a {trip_name} ski trip for a week. Here are the information on this trip:
1. I have booked airport parking: {airport_parking}
2. Booked flight: {flight_info}
3. Booked hotel in the destination: {hotel}
4. Booked car rental in the destination: {car_rental}
5. We will ski in the following Ski resorts in UT: Solitude Mountain Resort, Brighton Resort, and Alta Snowbird. 

With all above information, can you make a detailed day to day ski trip plan include following:
1. Entire trip planning starting from the every beginning, New York, like from parking, then fly out, then to SLC, then get Rental car, then go to hotel, 
then Ski activities (filling in for the entire stay in this trip), then check out hotel, return rental car, take flight back.
2. Each day should include driving directions and address to one of those Ski resorts, where to park, and where to eat dinner.
3. For any location, address, contact info are needed.

"""

print(prompt_text)


I am going to plan a Ski SLC ski trip for a week. Here are the information on this trip:
1. I have booked airport parking: The parking information is as follows:
{
  "address": "14707 Guy R Brewer Boulevard, New York, United States",
  "rating": "4",
  "reviews": "539",
  "type": "Commercial Parking Lot or Garage",
  "operating_hours": {
    "Sunday": "24 hours",
    "Monday": "24 hours",
    "Tuesday": "24 hours",
    "Wednesday": "24 hours",
    "Thursday": "24 hours",
    "Friday": "24 hours",
    "Saturday": "24 hours"
  }
}
2. Booked flight: The flight information is as follows:
{
  "flight1": {
    "departure": "New York-JFK",
    "arrival": "Salt Lake City",
    "date": "Dec 6, 2023",
    "departureTime": "11:00 am",
    "arrivalTime": "02:15 pm",
    "duration": "5 hrs 15 mins",
    "stops": "Nonstop",
    "class": "Blue Basic",
    "flightNumber": "B6871",
    "operatedBy": "JetBlue"
  },
  "flight2": {
    "departure": "Salt Lake City",
    "arrival": "New York-JFK",
    "da

In [28]:
prompt_file = './reports/report_prompt.txt'

with open(prompt_file, 'w') as file:
    file.write(prompt_text)